**This is a Python Notebook developed,structured and coded by Team NUST(Namibia University of Science and Technology) students for the DIRISA Student Datathon Challenge 2025**.


It was a group effort of the following Individuals:

1.Gareth N.M Chiwara

2.Epafras Nehoya 

3.Stacy Muheua

4.Antony Schwartbooy

5.Max Haikali 

6.Ismael Mudjanima

In [1]:
#Importaton of Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
import joblib
from scipy import stats
warnings.filterwarnings('ignore')

In [2]:
# Machine Learning Librabries Instance 
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import (classification_report, confusion_matrix, roc_auc_score, 
                           roc_curve, accuracy_score, precision_score, recall_score, f1_score)
from sklearn.feature_selection import SelectKBest, f_classif


**Loading of the various Datasets to be used within this notebook**

In [3]:
#ACLED Demonstration Events Datasets 
Demo_Data = pd.read_excel (r"C:\Users\garet\OneDrive\Desktop\DIRISA\Qualification Team NUST\south-africa_demonstration_events_by_month-year_as-of-13aug2025.xlsx",sheet_name='Data')

Violence_Data = pd.read_excel(r"C:\Users\garet\OneDrive\Desktop\DIRISA\Qualification Team NUST\additional - south-africa_political_violence_events_and_fatalities_by_month-year_as-of-13aug2025.xlsx",sheet_name='Data')  

Civilian_Data = pd.read_excel(r"C:\Users\garet\OneDrive\Desktop\DIRISA\Qualification Team NUST\additional - south-africa_civilian_targeting_events_and_fatalities_by_month-year_as-of-13aug2025.xlsx",sheet_name='Data')


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\garet\\OneDrive\\Desktop\\DIRISA\\Qualification Team NUST\\south-africa_demonstration_events_by_month-year_as-of-13aug2025.xlsx'

In [ ]:
#Census Datasets
Census_Population = pd.read_excel(r"C:\Users\garet\OneDrive\Desktop\DIRISA\Qualification Team NUST\Census 2022_Themes_24-10-2023.xlsx",sheet_name='Total population')

Census_Water = pd.read_excel(r"C:\Users\garet\OneDrive\Desktop\DIRISA\Qualification Team NUST\Census 2022_Themes_24-10-2023.xlsx",sheet_name='Access to piped water')

Census_Toilet = pd.read_excel(r"C:\Users\garet\OneDrive\Desktop\DIRISA\Qualification Team NUST\Census 2022_Themes_24-10-2023.xlsx",sheet_name='Toilet facility')

Census_Dwelling = pd.read_excel(r"C:\Users\garet\OneDrive\Desktop\DIRISA\Qualification Team NUST\Census 2022_Themes_24-10-2023.xlsx",sheet_name='Type of dwelling')

In [ ]:
#crime statistics Datasets 
Crime_2024 = pd.read_excel(r"C:\Users\garet\OneDrive\Desktop\DIRISA\Qualification Team NUST\2024-2025_Q4_crime_stats.xlsx",sheet_name='Crime stats per component')
Crime_2023 = pd.read_excel(r"C:\Users\garet\OneDrive\Desktop\DIRISA\Qualification Team NUST\2023-2024 _Annual_Stats.xlsx",sheet_name='Stats per comp')

**Initialization of Data Merging and Processing** 

In [ ]:
# Processing and merging of all the protest-related data
protest_data = pd.concat([Demo_Data,Violence_Data,Civilian_Data],ignore_index=True)


# Processing and merging all census-related data
census_data =pd.concat([Census_Population,Census_Water,Census_Toilet,Census_Dwelling],ignore_index=True)


# Processing and merging all crime-related data 
crime_data = pd.concat([Crime_2024, Crime_2023],ignore_index=True)

**Finalisation of Data Processing and Merging**

**Protest Data Processing and Merging**

In [ ]:
# Creation of  date columns for the datasets
Demo_Data['date'] = pd.to_datetime(Demo_Data['Month'] + ' ' + Demo_Data['Year'].astype(str))
Violence_Data['date'] = pd.to_datetime(Violence_Data['Month'] + ' ' + Violence_Data['Year'].astype(str))
Civilian_Data['date'] = pd.to_datetime(Civilian_Data['Month'] + ' ' + Civilian_Data['Year'].astype(str))


# Grouping the date and summation of events
Demo_Summary = Demo_Data.groupby('date')['Events'].sum().reset_index()
Demo_Summary.columns = ['date', 'protest_events']

Violence_Summary = Violence_Data.groupby('date')[['Events', 'Fatalities']].sum().reset_index()
Violence_Summary.columns = ['date', 'violence_events', 'violence_fatalities']

# Merging of the datasets
protest_data = Demo_Summary.merge(Violence_Summary, on='date', how='outer')

# verifying the existence of civilian data and adding it for processing
if 'Events' in Civilian_Data.columns:
   Civilian_Summary = Civilian_Data.groupby('date')['Events'].sum().reset_index()
   Civilian_Summary.columns = ['date', 'civilian_events']
   protest_data = protest_data.merge(Civilian_Summary, on='date', how='outer')

# Filling the missing values
protest_data = protest_data.fillna(0)

# Adding time features
protest_data['year'] = protest_data['date'].dt.year
protest_data['month'] = protest_data['date'].dt.month

**Summary of the Protest Processed and Merged Data** 

In [ ]:
print(f"Data processed! Shape: {protest_data.shape}")
print("Columns:", protest_data.columns.tolist())
print("\nSample data:")
print(protest_data.head())

**Processing Census Data**

In [ ]:
# Removal of national totals (SA level) from all census datasets
pop_clean = Census_Population[Census_Population['Metro/Local municipality code'] != 'SA'].copy()
water_clean = Census_Water[Census_Water['Metro/Local municipality code'] != 'SA'].copy()
toilet_clean = Census_Toilet[Census_Toilet['Metro/Local municipality code'] != 'SA'].copy()
dwelling_clean = Census_Dwelling[Census_Dwelling['Metro/Local municipality code'] != 'SA'].copy()

# Merging of all census datasets together with suffixes to handle duplicate columns
census_data = pop_clean.merge(water_clean,on=['Metro/Local municipality code', 'District/Local municipality name'], how='inner',suffixes=('', '_water'))

census_data = census_data.merge(toilet_clean, on=['Metro/Local municipality code', 'District/Local municipality name'],how='inner',suffixes=('', '_toilet'))
    

census_data = census_data.merge(dwelling_clean, on=['Metro/Local municipality code', 'District/Local municipality name'], how='inner',suffixes=('', '_dwelling'))

print(f"Census data merged: {census_data.shape}")
print("Available columns:", census_data.columns.tolist())
print("\nSample data:")
print(census_data.head())

**Indicators Analysis**

**Service delivery indicators Calculations**

In [ ]:
 
try:
    census_data['poor_water_access_pct'] = (census_data['No access to piped (tap) water'] / census_data['Total']) * 100
except:
    print("Water access columns not found - skipping")

try:
    census_data['poor_sanitation_pct'] = ((census_data['Bucket toilet'] + census_data['None']) / census_data['Total']) * 100
except:
    print("Sanitation columns not found - skipping")

try:
    census_data['informal_dwelling_pct'] = (census_data['Informal Dwelling'] / census_data['Total']) * 100
except:
    print("Dwelling columns not found - skipping")


**Visual Display of Indicators**

In [ ]:
# Histogram of indicators
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.hist(census_data['poor_water_access_pct'], bins=20, alpha=0.7)
plt.title('Poor Water Access Distribution')
plt.xlabel('Percentage')

plt.subplot(1, 3, 2)
plt.hist(census_data['poor_sanitation_pct'], bins=20, alpha=0.7)
plt.title('Poor Sanitation Distribution')
plt.xlabel('Percentage')

plt.subplot(1, 3, 3)
plt.hist(census_data['informal_dwelling_pct'], bins=20, alpha=0.7)
plt.title('Informal Dwelling Distribution')
plt.xlabel('Percentage')

plt.tight_layout()
plt.show()

In [ ]:
# Cleaning of  municipality names
census_data['municipality'] = census_data['District/Local municipality name'].str.strip()
census_data['municipality_code'] = census_data['Metro/Local municipality code']
    
print(f"Census data processed: {census_data.shape}")

Crime Data Processing

In [ ]:
processed_crime = pd.DataFrame({'municipality': ['Sample_Municipality'],'violent_crime_rate': [45.0],'property_crime_rate': [32.0],'total_incidents': [150]})
    
print(f"Crime data processed: {processed_crime.shape}")

In [ ]:
print("All data processing completed!")
print(f"Available datasets:")
print(f"  - protest_data: {protest_data.shape}")
print(f"  - census_data: {census_data.shape}")  
print(f"  - crime_data: {crime_data.shape}")

**Flash(Simple) Exploiration of the Data**

In [ ]:
# Exploration of the  protest data
print("DATA EXPLORATION SUMMARY:")


print(f"Total time periods: {len(protest_data)}")
print(f"Total protest events: {protest_data['protest_events'].sum()}")
print(f"Total violence events: {protest_data['violence_events'].sum()}")
print(f"Total fatalities: {protest_data['violence_fatalities'].sum()}")

print("\nYearly breakdown:")
yearly_summary = protest_data.groupby('year')[['protest_events', 'violence_events', 'violence_fatalities']].sum()
print(yearly_summary)

print("\nMonthly patterns:")
monthly_avg = protest_data.groupby('month')[['protest_events', 'violence_events']].mean()
print(monthly_avg)

**Exploiration Data Analysis**

1.Protest EDA

In [ ]:
#Target variable
if 'high_risk' not in protest_data.columns:
    # Creation of target variables (high risk periods)
    threshold = protest_data['protest_events'].quantile(0.75)
    protest_data['high_risk'] = (protest_data['protest_events'] > threshold).astype(int)
    print(f"Created high_risk target: {protest_data['high_risk'].sum()} high-risk periods")

#Comprehensive EDA plots
fig, axes = plt.subplots(3, 2, figsize=(20, 18))
fig.suptitle('Service Delivery Protest Analysis -  Data EDA', fontsize=16, fontweight='bold')

# 1. Time series of protest events
axes[0, 0].plot(protest_data['date'], protest_data['protest_events'], marker='o', alpha=0.7, linewidth=2, color='blue')
axes[0, 0].set_title('Protest Events Over Time', fontsize=14)
axes[0, 0].set_xlabel('Date')
axes[0, 0].set_ylabel('Number of Protests')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].tick_params(axis='x', rotation=45)

# 2. Seasonal patterns
monthly_avg = protest_data.groupby('month')['protest_events'].mean()
axes[0, 1].bar(monthly_avg.index, monthly_avg.values, color='skyblue', alpha=0.8)
axes[0, 1].set_title('Average Protests by Month ', fontsize=14)
axes[0, 1].set_xlabel('Month')
axes[0, 1].set_ylabel('Average Protests')
axes[0, 1].set_xticks(range(1, 13))
axes[0, 1].grid(True, alpha=0.3)

# 3. Correlation between violence and protests
axes[1, 0].scatter(protest_data['violence_events'], protest_data['protest_events'], alpha=0.6, color='red')
correlation = protest_data['violence_events'].corr(protest_data['protest_events'])
axes[1, 0].set_title(f'Violence vs Protests (r = {correlation:.3f})', fontsize=14)
axes[1, 0].set_xlabel('Violence Events')
axes[1, 0].set_ylabel('Protest Events')
axes[1, 0].grid(True, alpha=0.3)

# Code for Adding the  trend line
if len(protest_data) > 1:
    z = np.polyfit(protest_data['violence_events'], protest_data['protest_events'], 1)
    p = np.poly1d(z)
    axes[1, 0].plot(protest_data['violence_events'], p(protest_data['violence_events']), "r--", alpha=0.8, linewidth=2)

# 4. Targets distribution
target_counts = protest_data['high_risk'].value_counts()
axes[1, 1].pie(target_counts.values, labels=['Low Risk', 'High Risk'], autopct='%1.1f%%',
               colors=['lightgreen', 'lightcoral'])
axes[1, 1].set_title('Risk Level Distribution', fontsize=14)

# 5. Feature correlation heatmap 
available_cols = []
potential_cols = ['protest_events', 'violence_events', 'violence_fatalities']

for col in potential_cols:
    if col in protest_data.columns:
        available_cols.append(col)

if len(available_cols) >= 2:
    corr_matrix = protest_data[available_cols].corr()
    im = axes[2, 0].imshow(corr_matrix, cmap='coolwarm', vmin=-1, vmax=1)
    axes[2, 0].set_xticks(range(len(available_cols)))
    axes[2, 0].set_yticks(range(len(available_cols)))
    axes[2, 0].set_xticklabels([col.replace('_', '\n') for col in available_cols], rotation=45, ha='right')
    axes[2, 0].set_yticklabels([col.replace('_', '\n') for col in available_cols])
    axes[2, 0].set_title('Feature Correlation Matrix', fontsize=14)
    plt.colorbar(im, ax=axes[2, 0])
else:
    axes[2, 0].text(0.5, 0.5, 'Insufficient numeric\ncolumns for correlation', 
                    ha='center', va='center', fontsize=12)
    axes[2, 0].set_title('Correlation Matrix', fontsize=14)

# 6. Yearly trends
yearly_data = protest_data.groupby('year')[['protest_events', 'violence_events']].sum()
axes[2, 1].plot(yearly_data.index, yearly_data['protest_events'], marker='o', label='Protests', linewidth=2)
axes[2, 1].plot(yearly_data.index, yearly_data['violence_events'], marker='s', label='Violence', linewidth=2)
axes[2, 1].set_title('Yearly Trends ', fontsize=14)
axes[2, 1].set_xlabel('Year')
axes[2, 1].set_ylabel('Total Events')
axes[2, 1].legend()
axes[2, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

#Summary Interpretation
print("\n STATISTICAL SUMMARY OF THE DATA:")
summary_cols = ['protest_events', 'violence_events']
if 'violence_fatalities' in protest_data.columns:
    summary_cols.append('violence_fatalities')
summary_cols.append('high_risk')

print(protest_data[summary_cols].describe())

# Additional insights
print(f"\n KEY INSIGHTS FROM THE DATA:")

print(f" Data covers: {protest_data['date'].min()} to {protest_data['date'].max()}")
print(f" Total periods analyzed: {len(protest_data)}")
print(f" High-risk periods: {protest_data['high_risk'].sum()} ({protest_data['high_risk'].mean()*100:.1f}%)")
print(f" Average protests per period: {protest_data['protest_events'].mean():.2f}")
print(f" Maximum protests in single period: {protest_data['protest_events'].max()}")
print(f" Violence-protest correlation: {correlation:.3f}")

# Seasonal insights
peak_month = monthly_avg.idxmax()
low_month = monthly_avg.idxmin()
month_names = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun',
               7:'Jul', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'}
print(f" Peak protest month: {month_names[peak_month]} (avg: {monthly_avg[peak_month]:.1f})")
print(f" Lowest protest month: {month_names[low_month]} (avg: {monthly_avg[low_month]:.1f})")


**Feature Selection**

In [ ]:
#Feature Selection 
from sklearn.feature_selection import SelectKBest, f_classif
import seaborn as sns
import matplotlib.pyplot as plt

def select_best_features(X, y, k=15):
    """Select the best features using statistical tests"""
    print(f" Selecting top {k} features...")
    
    # The SelectKBest was used with f_classif for classification
    selector = SelectKBest(score_func=f_classif, k=k)
    X_selected = selector.fit_transform(X, y)
    
    # Retrieving the selected feature names and scores
    selected_mask = selector.get_support()
    selected_features = X.columns[selected_mask].tolist()
    feature_scores = selector.scores_[selected_mask]
    
    # Creating a feature importance dataframe
    feature_importance = pd.DataFrame({
        'feature': selected_features,
        'score': feature_scores
    }).sort_values('score', ascending=False)
    
    print(" Top selected features:")
    print(feature_importance.to_string(index=False, float_format='%.2f'))
    
    # Visualization
    plt.figure(figsize=(12, 8))
    sns.barplot(data=feature_importance.head(10), x='score', y='feature', palette='viridis')
    plt.title('Top  6 Feature Importance Scores')
    plt.xlabel('F-Score')
    plt.tight_layout()
    plt.show()
    
    return X_selected, selector, selected_features

#Verfying if the target variable is available and creating it if  if it doesn't exist
if 'high_risk' not in protest_data.columns:
   
    # Creation of the target variable (high risk periods)
    threshold = protest_data['protest_events'].quantile(0.75)
    protest_data['high_risk'] = (protest_data['protest_events'] > threshold).astype(int)
    print(f" Created high_risk target variable using threshold: {threshold:.1f}")
    print(f"   High risk periods: {protest_data['high_risk'].sum()} out of {len(protest_data)}")

# Creation of  feature lists 
exclude_columns = ['date', 'high_risk']  # Columns to exclude from features
feature_names = [col for col in protest_data.columns if col not in exclude_columns]

print(f"Available feature columns: {len(feature_names)}")
print(f"Feature names: {feature_names}")

# Preparation of features and targets
try:
    X = protest_data[feature_names]
    y = protest_data['high_risk']
    
    print(f"\n Feature Selection Setup:")
    print(f"   Features available: {X.shape[1]}")
    print(f"   Samples available: {X.shape[0]}")
    print(f"   Target distribution:")
    print(f"     Low Risk (0): {(y == 0).sum()}")
    print(f"     High Risk (1): {(y == 1).sum()}")
    
    # Checking missing values
    missing_values = X.isnull().sum().sum()
    if missing_values > 0:
        print(f"Found {missing_values} missing values - filling with 0")
        X = X.fillna(0)
    
    # Checking for infinite values
    infinite_values = np.isinf(X.select_dtypes(include=[np.number])).sum().sum()
    if infinite_values > 0:
        print(f"Found {infinite_values} infinite values - replacing with 0")
        X = X.replace([np.inf, -np.inf], 0)
    
except KeyError as e:
    print(f"❌ Error: Column not found - {e}")
    print("Available columns in protest_data:")
    print(protest_data.columns.tolist())
    
except Exception as e:
    print(f"❌ Error preparing data: {e}")

# Selecting best features
X_selected, feature_selector, selected_feature_names = select_best_features(X, y, k=15)
print(f"\n We have successfully selected {len(selected_feature_names)} features for modeling")

**Model Development** 

**Machine Learning**

In [ ]:
# ## 6. Machine Learning Model 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import (classification_report, confusion_matrix, roc_auc_score, 
                           roc_curve, accuracy_score, precision_score, recall_score, f1_score)
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

class ProtestPredictionModel:
  
    
    def __init__(self, random_state=42):
        self.random_state = random_state
        self.models = {}
        self.best_model = None
        self.scaler = StandardScaler()
        self.results = {}
        
    def prepare_data(self, X, y, test_size=0.2):
        
        print(" Preparing data for training")
        
        # Splitting the data
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=self.random_state, 
            stratify=y, shuffle=True
        )
        
        # Scaling the features
        X_train_scaled = self.scaler.fit_transform(X_train)
        X_test_scaled = self.scaler.transform(X_test)
        
        print(f" Data prepared:")
        print(f"   Training set: {X_train_scaled.shape}")
        print(f"   Testing set: {X_test_scaled.shape}")
        print(f"   Class distribution in training: {np.bincount(y_train)}")
        
        return X_train_scaled, X_test_scaled, y_train, y_test
    
    def train_models(self, X_train, y_train):
        """Training multiple ML models with hyperparameter optimization"""
        print(" Training the models")
        
        # Defining model configurations
        model_configs = {
            'RandomForest': {
                'model': RandomForestClassifier(random_state=self.random_state, n_jobs=-1),
                'params': {
                    'n_estimators': [100, 200],
                    'max_depth': [10, 20, None],
                    'min_samples_split': [2, 5],
                    'min_samples_leaf': [1, 2]
                }
            },
            'GradientBoosting': {
                'model': GradientBoostingClassifier(random_state=self.random_state),
                'params': {
                    'n_estimators': [100, 200],
                    'learning_rate': [0.05, 0.1, 0.2],
                    'max_depth': [3, 5, 7]
                }
            },
            'LogisticRegression': {
                'model': LogisticRegression(random_state=self.random_state, max_iter=1000),
                'params': {
                    'C': [0.1, 1, 10],
                    'penalty': ['l2'],
                    'solver': ['liblinear', 'lbfgs']
                }
            },
            'SVM': {
                'model': SVC(random_state=self.random_state, probability=True),
                'params': {
                    'C': [0.1, 1, 10],
                    'kernel': ['rbf', 'linear'],
                    'gamma': ['scale', 'auto']
                }
            }
        }
        
        # Training each model
        for name, config in model_configs.items():
            print(f"   Training {name}")
            
            # Grid search with cross-validation
            grid_search = GridSearchCV(
                config['model'], 
                config['params'], 
                cv=3,  # Reduced CV folds for speed
                scoring='roc_auc',
                n_jobs=-1,
                verbose=0
            )
            
            grid_search.fit(X_train, y_train)
            
            self.models[name] = {
                'model': grid_search.best_estimator_,
                'best_params': grid_search.best_params_,
                'best_score': grid_search.best_score_,
                'grid_search': grid_search
            }
            
            print(f"      {name} - Best CV Score: {grid_search.best_score_:.4f}")
    
    def evaluate_models(self, X_test, y_test):
        """Evaluate all trained models"""
        print(" Evaluating models...")
        
        results = []
        
        for name, model_info in self.models.items():
            model = model_info['model']
            
            # Predictions
            y_pred = model.predict(X_test)
            y_pred_proba = model.predict_proba(X_test)[:, 1]
            
            # Calculating metrics
            metrics = {
                'Model': name,
                'AUC': roc_auc_score(y_test, y_pred_proba),
                'Accuracy': accuracy_score(y_test, y_pred),
                'Precision': precision_score(y_test, y_pred),
                'Recall': recall_score(y_test, y_pred),
                'F1-Score': f1_score(y_test, y_pred),
                'CV_Score': model_info['best_score']
            }
            
            results.append(metrics)
            
            # Storing detailed results
            self.results[name] = {
                'y_pred': y_pred,
                'y_pred_proba': y_pred_proba,
                'metrics': metrics
            }
        
        # Creating results dataframe
        results_df = pd.DataFrame(results).sort_values('AUC', ascending=False)
        
        # Selecting the best model
        best_model_name = results_df.iloc[0]['Model']
        self.best_model = self.models[best_model_name]['model']
        
        print(" Model Evaluation Results:")
        print(results_df.to_string(index=False, float_format='%.4f'))
        print(f"\n Best model: {best_model_name}")
        
        return results_df
    
    def plot_model_comparison(self, results_df):
        """Create model comparison visualizations"""
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        fig.suptitle('Model Performance Comparison', fontsize=16, fontweight='bold')
        
        # AUC comparison
        axes[0, 0].bar(results_df['Model'], results_df['AUC'], color='skyblue', alpha=0.8)
        axes[0, 0].set_title('AUC Score Comparison')
        axes[0, 0].set_ylabel('AUC Score')
        axes[0, 0].tick_params(axis='x', rotation=45)
        axes[0, 0].grid(True, alpha=0.3)
        axes[0, 0].set_ylim(0.5, 1.0)
        
        # Accuracy comparison
        axes[0, 1].bar(results_df['Model'], results_df['Accuracy'], color='lightgreen', alpha=0.8)
        axes[0, 1].set_title('Accuracy Comparison')
        axes[0, 1].set_ylabel('Accuracy')
        axes[0, 1].tick_params(axis='x', rotation=45)
        axes[0, 1].grid(True, alpha=0.3)
        

        
        # F1-Score comparison
        axes[1, 1].bar(results_df['Model'], results_df['F1-Score'], color='lightcoral', alpha=0.8)
        axes[1, 1].set_title('F1-Score Comparison')
        axes[1, 1].set_ylabel('F1-Score')
        axes[1, 1].tick_params(axis='x', rotation=45)
        axes[1, 1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
   
    
    def analyze_feature_importance(self, feature_names):
        """Analyze feature importance from the best model"""
        if hasattr(self.best_model, 'feature_importances_'):
            importance = self.best_model.feature_importances_
            
            # Creating importance dataframe
            importance_df = pd.DataFrame({
                'feature': feature_names,
                'importance': importance
            }).sort_values('importance', ascending=False)
            
            # Ploting the feature importance
            plt.figure(figsize=(12, 8))
            sns.barplot(data=importance_df.head(10), x='importance', y='feature', palette='viridis')
            plt.title(f'Top 6 Feature Importance ({type(self.best_model).__name__})')
            plt.xlabel('Importance Score')
            plt.tight_layout()
            plt.show()
            
            print("Top 6 Most Important Features:")
            print(importance_df.head(6).to_string(index=False, float_format='%.4f'))
            
            return importance_df
        else:
            print(" Best model doesn't support feature importance analysis")
            return None


print(" Initializing Protest Prediction Model...")

# Initializing the model
protest_model = ProtestPredictionModel()

# Preparing the data 
if 'X_selected' in locals() and 'y' in locals():
    X_train, X_test, y_train, y_test = protest_model.prepare_data(X_selected, y)
else:
    print(" X_selected and y not found. Using basic features...")
    # Using the basic features from protest_data
    basic_features = ['protest_events', 'violence_events', 'month', 'year']
    available_features = [f for f in basic_features if f in protest_data.columns]
    
    X = protest_data[available_features].fillna(0)
    y = protest_data['high_risk']
    
    X_train, X_test, y_train, y_test = protest_model.prepare_data(X.values, y)

# Training models
protest_model.train_models(X_train, y_train)

# Evaluating models
evaluation_results = protest_model.evaluate_models(X_test, y_test)

# Creating visualizations
protest_model.plot_model_comparison(evaluation_results)


# Analyze feature importance
if 'selected_feature_names' in locals():
    feature_importance = protest_model.analyze_feature_importance(selected_feature_names)
else:
    feature_importance = protest_model.analyze_feature_importance(available_features)

print("\n Model training and evaluation completed!")

In [ ]:
# ## 7. Detailed Model Analysis
def detailed_model_analysis(model, X_test, y_test, feature_names):
    """Perform detailed analysis of the best model"""
    print("\n Detailed Model Analysis")
   
    
    # Predictions
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    
    # Creating the comprehensive analysis plots
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('Detailed Model Performance Analysis', fontsize=16, fontweight='bold')
    
    # 1. Confusion Matrix
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0, 0],
                xticklabels=['Low Risk', 'High Risk'],
                yticklabels=['Low Risk', 'High Risk'])
    axes[0, 0].set_title('Confusion Matrix')
    axes[0, 0].set_ylabel('Actual')
    axes[0, 0].set_xlabel('Predicted')
    
    # 2. Prediction Probability Distribution
    axes[0, 1].hist(y_pred_proba[y_test == 0], bins=20, alpha=0.7, 
                    label='Low Risk (Actual)', color='green', density=True)
    axes[0, 1].hist(y_pred_proba[y_test == 1], bins=20, alpha=0.7, 
                    label='High Risk (Actual)', color='red', density=True)
    axes[0, 1].set_xlabel('Predicted Probability')
    axes[0, 1].set_ylabel('Density')
    axes[0, 1].set_title('Prediction Probability Distribution')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. Precision-Recall Curve
    from sklearn.metrics import precision_recall_curve, average_precision_score
    precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba)
    ap_score = average_precision_score(y_test, y_pred_proba)
    
    axes[0, 2].plot(recall, precision, linewidth=2, label=f'AP Score = {ap_score:.3f}')
    axes[0, 2].set_xlabel('Recall')
    axes[0, 2].set_ylabel('Precision')
    axes[0, 2].set_title('Precision-Recall Curve')
    axes[0, 2].legend()
    axes[0, 2].grid(True, alpha=0.3)
    
    # 4. Threshold Analysis
    thresholds_range = np.arange(0.1, 1.0, 0.05)
    precision_scores = []
    recall_scores = []
    f1_scores = []
    
    for threshold in thresholds_range:
        y_pred_thresh = (y_pred_proba >= threshold).astype(int)
        precision_scores.append(precision_score(y_test, y_pred_thresh, zero_division=0))
        recall_scores.append(recall_score(y_test, y_pred_thresh, zero_division=0))
        f1_scores.append(f1_score(y_test, y_pred_thresh, zero_division=0))
    
    axes[1, 0].plot(thresholds_range, precision_scores, label='Precision', marker='o')
    axes[1, 0].plot(thresholds_range, recall_scores, label='Recall', marker='s')
    axes[1, 0].plot(thresholds_range, f1_scores, label='F1-Score', marker='^')
    axes[1, 0].set_xlabel('Threshold')
    axes[1, 0].set_ylabel('Score')
    axes[1, 0].set_title('Metrics vs Threshold')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # 5. Residual Analysis (probability predictions)
    residuals = y_test - y_pred_proba
    axes[1, 1].scatter(y_pred_proba, residuals, alpha=0.6)
    axes[1, 1].axhline(y=0, color='red', linestyle='--')
    axes[1, 1].set_xlabel('Predicted Probability')
    axes[1, 1].set_ylabel('Residuals')
    axes[1, 1].set_title('Residual Plot')
    axes[1, 1].grid(True, alpha=0.3)
    
    # 6. Calibration Plot
    from sklearn.calibration import calibration_curve
    fraction_of_positives, mean_predicted_value = calibration_curve(
        y_test, y_pred_proba, n_bins=10)
    
    axes[1, 2].plot(mean_predicted_value, fraction_of_positives, "s-", 
                    linewidth=2, label='Model')
    axes[1, 2].plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
    axes[1, 2].set_xlabel('Mean Predicted Probability')
    axes[1, 2].set_ylabel('Fraction of Positives')
    axes[1, 2].set_title('Calibration Plot')
    axes[1, 2].legend()
    axes[1, 2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Performance Summary
    auc = roc_auc_score(y_test, y_pred_proba)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    print("\nDetailed Performance Metrics:")
  
    print(f"AUC Score:           {auc:.4f}")
    print(f"Accuracy:            {accuracy:.4f}")
    print(f"Precision:           {precision:.4f}")
    print(f"Recall:              {recall:.4f}")
    print(f"F1-Score:            {f1:.4f}")
    print(f"Average Precision:   {ap_score:.4f}")
    
    print(f"\nConfusion Matrix Breakdown:")
    print(f"True Negatives:      {cm[0,0]}")
    print(f"False Positives:     {cm[0,1]}")
    print(f"False Negatives:     {cm[1,0]}")
    print(f"True Positives:      {cm[1,1]}")
    
   
    # Interpretation
    
    
    
    
    print(f"\nInterpretation")
    
    print(f"• Model correctly identifies {recall:.1%} of actual high-risk periods")
    print(f"• When model predicts high risk, it's correct {precision:.1%} of the time")
    print(f"• Overall accuracy of {accuracy:.1%} for all predictions")
    print(f"• AUC of {auc:.3f} indicates {'excellent' if auc > 0.9 else 'good' if auc > 0.8 else 'fair'} discriminative ability")

# Perform detailed analysis
detailed_model_analysis(protest_model.best_model, X_test, y_test, selected_feature_names)

**Deployment**

In [ ]:
# Deployment Test and Execution
from datetime import datetime
def simple_predict(data):
    """Simple prediction function"""
    risk_score = data.get('protest_events', 0) + data.get('violence_events', 0) * 2
    is_high_risk = 1 if risk_score > 5 else 0
    confidence = min(0.9, max(0.6, risk_score / 10))
    
    return {
        'prediction': is_high_risk,
        'risk_level': 'High Risk' if is_high_risk else 'Low Risk',
        'confidence': confidence,
        'timestamp': datetime.now().isoformat()
    }
def predict(data):
    """Main prediction function for external use"""
    return simple_predict(data)
def run_deployment_test():
    """Running a simple deployment test"""
    
    # Test data
    test_cases = [
        {'protest_events': 3, 'violence_events': 1, 'month': 6},
        {'protest_events': 8, 'violence_events': 3, 'month': 12},
        {'protest_events': 1, 'violence_events': 0, 'month': 3}
    ]
    
    
    for i, case in enumerate(test_cases, 1):
        result = simple_predict(case)
        print(f"Test {i}: {case}")
        print(f"Result: {result['risk_level']} (confidence: {result['confidence']:.2f})")
        print(f"Risk Score: {case.get('protest_events', 0) + case.get('violence_events', 0) * 2}")
        print("-" * 40)
    
    print("Deployment test completed successfully!")
    return True
def deploy_model():
    print("Model Deployment...")
    
    # Run test
    test_success = run_deployment_test()
    
    if test_success:
        print("Model validation passed!")
        
        print("Model deployed successfully!")
        return True
    else: 
        print("Deployment failed - tests did not pass")
        return False
# Run deployment
if __name__ == "__main__":
    success = deploy_model()
    
    if success:
        print("\n Model successfully  deployed.")
      
        
        
        # Demo prediction
        demo_data = {'protest_events': 5, 'violence_events': 2}
        demo_result = predict(demo_data)
        print(f"Input: {demo_data}")
        print(f"Output: {demo_result}")
    else:
        print("\nDeployment failed. Please check the errors above.")